### Implementing this paper: https://web.stanford.edu/~hastie/Papers/gap.pdf

In [1]:
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
import itertools

output_notebook()

Loading BokehJS ...

In [2]:
# helper functions
def plot_pca(X):
    pca = PCA(n_components=2, random_state=0)
    pca.fit(X.transpose())
    x1, x2 = pca.components_
    variance_explained = sum(pca.explained_variance_ratio_) *100
    title="first two components explained {0: .1f} % of variance".format(variance_explained)
    p = figure(title=title)
    p.scatter(x1, x2)
    show(p)
    
def calculate_wk(X, y):
    return sum([mean_ss(X[y==i]) for i in set(y)])

def mean_ss(points):
    """ averaged distances between every two data points"""
    s = sum([euclidean(p1, p2) for p1, p2 in itertools.combinations(points, r=2)])
    return s * 1.0/(2*len(points))

In [3]:
# simulate data points with 3 clusters
X_cluster, y_cluster = make_blobs(n_samples=1000, n_features=5, centers=4)
plot_pca(X_cluster)

In [5]:
# simulate data with isotropic gaussian distribution
X_normal = np.random.normal(size=(1000, 5))
plot_pca(X_normal)

In [8]:
# simulate data with uniform distribution
X_uniform = np.random.uniform(size=(1000, 5), high=3, low=-3)
plot_pca(X_uniform)

In [10]:
Wk = []
E_Wk_uniform = []
E_Wk_normal = []
RANGE = range(1, 11)
for n_cluster in RANGE:
    y_obs = KMeans(n_clusters=n_cluster).fit_predict(X_cluster)
    Wk.append(calculate_wk(X_cluster, y_obs))
    y_uniform = KMeans(n_clusters=n_cluster).fit_predict(X_uniform)
    E_Wk_uniform.append(calculate_wk(X_uniform, y_uniform)) 
    y_normal = KMeans(n_clusters=n_cluster).fit_predict(X_normal)
    E_Wk_normal.append(calculate_wk(X_normal, y_normal)) 

In [17]:
gap_uniform = np.log(np.array(E_Wk_uniform)) - np.log(np.array(Wk))
gap_normal = np.log(np.array(E_Wk_normal)) - np.log(np.array(Wk))
p = figure()
p.line(RANGE, gap_uniform, line_color="red", legend="null reference: uniform distribution")
p.square(RANGE, gap_uniform)
p.line(RANGE, gap_normal, line_color="blue", legend="null reference: gaussian distribution")
p.square(RANGE, gap_normal)
p.legend.location = "bottom_right"
p.xaxis.axis_label="Number of Clusters"
p.yaxis.axis_label="Gap statistics score"
show(p)